In [8]:
import numpy as np

## Globals
α = 2.79 #1/m
θs = 0.385
θr = 0.012
n = 7.26
m = 1 - 1/n
Ks = 2.07E-4  #m/s
qtarget = 3.32E-6 #m/s

def vanGenuchten(h:np.array) -> np.array:
    θe = np.where(
        h >= 0,
        1.0,
        np.power(1 + np.power(α*h*np.sign(h), n), -m)
    )
    return θe

def waterSaturation(h:np.array) -> np.array:
    θe = vanGenuchten(h)
    θ = θe * (θs - θr) + θr
    return θ

def mualemPermeability(h:np.array) -> np.array:
    θe = vanGenuchten(h)
    kr = np.sqrt(θe) * np.power(1-np.power(1-np.power(θe, 1/m), m), 2)
    return kr

def capillarity(h) -> np.array:
    x = np.where(
        h >= 0,
        0.0,
        α*m*n * np.power(α*h*np.sign(h), n-1) * np.power(1 + np.power(α*h*np.sign(h), n), -m-1)
        )
    x *= (θs - θr)
    return x

In [9]:
def my_bisection(f, a, b, tol): 
    # approximates a root, R, of f bounded 
    # by a and b to within tolerance 
    # | f(m) | < tol with m the midpoint 
    # between a and b Recursive implementation
    
    # check if a and b bound a root
    if np.sign(f(a)) == np.sign(f(b)):
        raise Exception(
         "The scalars a and b do not bound a root")
        
    # get midpoint
    m = (a + b)/2
    
    if np.abs(f(m)) < tol:
        # stopping condition, report m as root
        return m
    elif np.sign(f(a)) == np.sign(f(m)):
        # case where m is an improvement on a. 
        # Make recursive call with a = m
        return my_bisection(f, m, b, tol)
    elif np.sign(f(b)) == np.sign(f(m)):
        # case where m is an improvement on b. 
        # Make recursive call with b = m
        return my_bisection(f, a, m, tol)

In [48]:
kn = 0.016039
def hydraulic_cond(h):
    return qtarget - Ks * mualemPermeability(h) * kn

my_bisection(hydraulic_cond, -10, 0, 1e-12)

-0.057525634765625